# SITCOM-989 - M1M3 Inertia Compensation Performance

We need plots and metrics to evaluate the performance of the M1M3 Inertia Compensation System as described in [SITCOM-989].    
Examples of plots are:

* Hardpoint Load Cell Forces Minima and Maxima during slews as a function of time.
* Correlate the plots above with accelerations, velocities, and positions.
* (any other ideas?)

Petr asked to analyse the data obtained when slewing the telescope around 80 deg in elevation with and without inertia forces. 
The two datasets below that he used as an example contain movement from -100 deg in azimuth to 100 deg in a single slew. 
On both cases, we are using 30% motion settings in azimuth. 

* [M1M3 TMA Inertial forces Chronograph Dashboard on 2023-08-02 22:02 - 2023-08-02 22:04 UTC]
* [M1M3 TMA Inertial forces Chronograph Dashboard on 2023-07-28 02:15 - 2023-07-28 02:17 UTC]

Added a new dataset containing similar data but with 50% azimuth motion settinds. 

* [M1M3 TMA Inertial forces Chronograph Dashboard on 2023-08-03 03:20 - 2023-08-03 03:22 UTC]

[SITCOM-989]: https://jira.lsstcorp.org/browse/SITCOM-989


[M1M3 TMA Inertial forces Chronograph Dashboard on 2023-08-02 22:02 - 2023-08-02 22:04 UTC]: https://summit-lsp.lsst.codes/chronograf/sources/1/dashboards/252?redirect=%2Flogin%3Fredirect%3D%252Fsources%252F1%252Fdashboards%252F252%253Frefresh%253D30s%2526tempVars%255BDownsample%255D%253DDefault%2526tempVars%255BFunction%255D%253Draw%2526lower%253Dnow%2528%2529%252520-%25252015m%2526zoomedLower%253D2023-08-02T21%25253A23%25253A19.366Z%2526zoomedUpper%253D2023-08-02T21%25253A23%25253A23.843Z&refresh=Paused&tempVars%5BDownsample%5D=Default&tempVars%5BFunction%5D=mean%28%29&lower=2023-08-02T20%3A00%3A00.000Z&upper=2023-08-03T02%3A00%3A00.000Z&zoomedLower=2023-08-02T22%3A02%3A24.799Z&zoomedUpper=2023-08-02T22%3A04%3A02.450Zhttps://summit-lsp.lsst.codes/chronograf/sources/1/dashboards/252?redirect=%2Flogin%3Fredirect%3D%252Fsources%252F1%252Fdashboards%252F252%253Frefresh%253D30s%2526tempVars%255BDownsample%255D%253DDefault%2526tempVars%255BFunction%255D%253Draw%2526lower%253Dnow%2528%2529%252520-%25252015m%2526zoomedLower%253D2023-08-02T21%25253A23%25253A19.366Z%2526zoomedUpper%253D2023-08-02T21%25253A23%25253A23.843Z&refresh=Paused&tempVars%5BDownsample%5D=Default&tempVars%5BFunction%5D=mean%28%29&lower=2023-08-02T20%3A00%3A00.000Z&upper=2023-08-03T02%3A00%3A00.000Z&zoomedLower=2023-08-02T22%3A02%3A24.799Z&zoomedUpper=2023-08-02T22%3A04%3A02.450Z


[M1M3 TMA Inertial forces Chronograph Dashboard on 2023-07-28 02:15 - 2023-07-28 02:17 UTC]:https://summit-lsp.lsst.codes/chronograf/sources/1/dashboards/252?redirect=%2Flogin%3Fredirect%3D%252Fsources%252F1%252Fdashboards%252F252%253Frefresh%253D30s%2526tempVars%255BDownsample%255D%253DDefault%2526tempVars%255BFunction%255D%253Draw%2526lower%253Dnow%2528%2529%252520-%25252015m%2526zoomedLower%253D2023-08-02T21%25253A23%25253A19.366Z%2526zoomedUpper%253D2023-08-02T21%25253A23%25253A23.843Z&refresh=Paused&tempVars%5BDownsample%5D=Default&tempVars%5BFunction%5D=mean%28%29&lower=2023-07-28T02%3A00%3A00.000Z&upper=2023-07-28T03%3A30%3A00.000Z&zoomedLower=2023-07-28T02%3A15%3A45.730Z&zoomedUpper=2023-07-28T02%3A17%3A11.966Z

[M1M3 TMA Inertial forces Chronograph Dashboard on 2023-08-03 03:20 - 2023-08-03 03:22 UTC]:https://summit-lsp.lsst.codes/chronograf/sources/1/dashboards/252?redirect=%2Flogin%3Fredirect%3D%252Fsources%252F1%252Fdashboards%252F252%253Frefresh%253D30s%2526tempVars%255BDownsample%255D%253DDefault%2526tempVars%255BFunction%255D%253Draw%2526lower%253Dnow%2528%2529%252520-%25252015m%2526zoomedLower%253D2023-08-02T21%25253A23%25253A19.366Z%2526zoomedUpper%253D2023-08-02T21%25253A23%25253A23.843Z&refresh=Paused&tempVars%5BDownsample%5D=5Hz&tempVars%5BFunction%5D=mean%28%29&lower=2023-08-03T03%3A20%3A00.000Z&upper=2023-08-03T03%3A22%3A00.000Z

## Notebook Preparation

In [ ]:
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
from astropy.time import Time
from lsst.summit.testing.analysis.utils import create_logger
from lsst.summit.testing.analysis.m1m3 import inertia_compensation_system as m1m3_ics
from lsst.summit.testing.analysis.m1m3.plots import inertia_compensation_system as m1m3_ics_plots
from lsst.summit.utils.tmaUtils import TMAEvent, TMAEventMaker

## Data Collection

## Create Event Maker

We want to create a single instance of the `TMAEventMaker` object.  
Each instance might be quite heavy. 

In [ ]:
event_maker = TMAEventMaker()

## Example 1: Analyze M1M3 ICS per Slew Event

In [ ]:
# Az Slew from -100 to 100 with 80 El, 30% motion settings and Inertia Compensation On
begin = "2023-08-02T22:02:30"
end = "2023-08-02T22:04:00"

time_begin = Time(begin, format="isot", scale="utc")
time_end = Time(end, format="isot", scale="utc")
time_half = time_begin + (time_end - time_begin) * 0.5

event = event_maker.findEvent(time_half)
print(
    f"Slew happened from {begin=} to {end=} "
    f"and has sequence number {event.seqNum} "
    f"and observation day {event.dayObs}"
)

dataset1 = m1m3_ics.M1M3ICSAnalysis(event, event_maker.client)
fig1 = m1m3_ics_plots.plot_hp_measured_data(dataset1, log=dataset1.log, figsize=(18, 12))

In [ ]:
# Dataset 2 - Az Slew from -100 to 100 with 80 El, 30% motion settings and Inertia Compensation On
begin = "2023-07-28T02:17:15" 
end = "2023-07-28T02:17:55"

time_begin = Time(begin, format="isot", scale="utc")
time_end = Time(end, format="isot", scale="utc")
time_half = time_begin + (time_end - time_begin) * 0.5

event = event_maker.findEvent(time_half)
print(
    f"Slew happened from {begin=} to {end=} "
    f"and has sequence number {event.seqNum} "
    f"and observation day {event.dayObs}"
)

dataset2 = m1m3_ics.M1M3ICSAnalysis(event, event_maker.client)
fig2 = m1m3_ics_plots.plot_hp_measured_data(dataset2)

In [ ]:
# Dataset 3 - Az Slew from -100 to 100 with 80 El, 50% Az motion settings and Inertia Compensation On
begin = "2023-08-03T03:20:30"
end = "2023-08-03T03:21:20"

time_begin = Time(begin, format="isot", scale="utc")
time_end = Time(end, format="isot", scale="utc")
time_half = time_begin + (time_end - time_begin) * 0.5

event = event_maker.findEvent(time_half)
print(
    f"Slew happened from {begin=} to {end=} "
    f"and has sequence number {event.seqNum} "
    f"and observation day {event.dayObs}"
)

dataset3 = m1m3_ics.M1M3ICSAnalysis(event, event_maker.client)
m1m3_ics_plots.plot_hp_measured_data(dataset3, dpi=120)

## Example 2: Analyze M1M3 ICS per Day Obs 

In [ ]:
results = m1m3_ics.evaluate_m1m3_ics_day_obs(20230802, event_maker)